<a href="https://colab.research.google.com/github/YuliantiOenang/tryingHuggingFace/blob/main/test_multitraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("poem_sentiment")
dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 6.9 MB/s 
     |████████████████████████████████| 115 kB 50.4 MB/s 
     |████████████████████████████████| 212 kB 51.4 MB/s 
     |████████████████████████████████| 163 kB 45.1 MB/s 
     |████████████████████████████████| 95 kB 3.4 MB/s 
     |████████████████████████████████| 127 kB 52.5 MB/s 
     |████████████████████████████████| 115 kB 49.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

Dataset poem_sentiment downloaded and prepared to /root/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.5 MB/s 
     |████████████████████████████████| 7.6 MB 30.6 MB/s 


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
dataset['train'][0]

{'id': 0,
 'verse_text': 'with pale blue berries. in these peaceful shades--',
 'label': 1}

In [19]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

training_text = '<poem_contain>'+dataset['train'][0]['verse_text'] + ' <label>'+str(dataset['train'][0]['label'])
for i in range(1, 30):
  training_text += ' <poem_contain>'+dataset['train'][i]['verse_text'] +' <label>'+str(dataset['train'][i]['label'])

training_text

'<poem_contain>with pale blue berries. in these peaceful shades-- <label>1 <poem_contain>it flows so long as falls the rain, <label>2 <poem_contain>and that is why, the lonesome day, <label>0 <poem_contain>when i peruse the conquered fame of heroes, and the victories of mighty generals, i do not envy the generals, <label>3 <poem_contain>of inward strife for truth and liberty. <label>3 <poem_contain>the red sword sealed their vows! <label>3 <poem_contain>and very venus of a pipe. <label>2 <poem_contain>who the man, who, called a brother. <label>2 <poem_contain>and so on. then a worthless gaud or two, <label>0 <poem_contain>to hide the orb of truth--and every throne <label>2 <poem_contain>the call\'s more urgent when he journeys slow. <label>2 <poem_contain>with the _quart d\'heure_ of rabelais! <label>2 <poem_contain>and match, and bend, and thorough-blend, in her colossal form and face. <label>2 <poem_contain>have i played in different countries. <label>2 <poem_contain>tells us that th

In [ ]:
%%time
count_true = 0
predict = 0

preds = []
labels = []
for i in range(0, 20):
  new_data = training_text
  new_data += ' <poem_contain>'+dataset['test'][i]['verse_text']+' <label>'

  result = generator(new_data, max_length=100, num_return_sequences=5)

  label_index = result[0]['generated_text'].rfind('<label>')
  result_index = label_index+7
  pred = result[0]['generated_text'][result_index:result_index+1]
  preds.append(pred)
  
  label = str(dataset['test'][i]['label'])
  labels.append(label)
  if pred == label:
    count_true+=count_true+1


accuracy = count_true/len(preds)
print(accuracy)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 648, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 656, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 652, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 645, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is

In [12]:
print(preds)
print(labels)

['1', '1', '3', '2', '3', '4', '4', '3', '1', '1']
['2', '1', '2', '2', '2', '0', '0', '2', '2', '2']
